In [1]:
import os
import sys
os.chdir('..')
sys.path.append(os.getcwd())

In [2]:
import torch
from Models.pretrain import *
from Models.cPB import cPB
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pickle
import argparse
from torch.autograd import Variable
import pprint
import copy
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=UserWarning, module='torch.storage')
warnings.filterwarnings("ignore", category=DeprecationWarning, module='torch.storage')

# SINE datasets


In [3]:
base_model = 'GRU'
batch_size = 128
hidden_size = 250
seq_len = 10
epoch_size=10
lr = 0.01
number_of_tasks=4
mask_selection_NofBatch = 50
input_size=2
iteration=1
mask_init='uniform'
dataset='SINE'
dataset_name = "sine_rw10_mode5_extended_16-16_1234"
Pretrain_task = '1'

df = pd.read_csv(os.path.join(f"datasets/{dataset}/", f"{dataset_name}.csv"))
pretrain_model_addr=f'Performance/Pretrain/GRU/{dataset}/After/sine-6_6-1234-t{Pretrain_task}-{base_model}-pretrain-hidden{hidden_size}-epoch10_itter{iteration}.pickle'
mask_weights=[] #if we have initial masks then reload it here



In [ ]:
model=cPB(lr = lr ,hidden_size=hidden_size, seq_len=seq_len,base_model=base_model, pretrain_model_addr=pretrain_model_addr,
           mask_weights=mask_weights, mask_init=mask_init, number_of_tasks=number_of_tasks,epoch_size=epoch_size,input_size=input_size)


for iter in range(0,iteration):
  # loop for each task
  for task in range(1, df["task"].max() + 1):
    df_task = df[df["task"] == task]
    df_task = df_task.drop(columns="task")
    # loop based on each batch of data
    batch_cont=0
    model.weights_copy(task)
    for i in range(0, len(df_task), batch_size):
      x = df_task.iloc[i : i + batch_size, 0:-1].values.astype(np.float32)
      y = list(df_task.iloc[i : i + batch_size, -1])
      if batch_cont<mask_selection_NofBatch:
        for mask_index in range (0, task):
          model.predict_many(x,y,mask_index,task,mask_selection=True)
          model.learn_many(x,y,mask_index)
        batch_cont+=1
      elif batch_cont==mask_selection_NofBatch:
        best_mask_index=model.add_new_column(task)
        model.predict_many(x,y,best_mask_index,task,mask_selection=False)
        model.learn_many(x,y,best_mask_index)
        batch_cont+=1
      elif batch_cont>mask_selection_NofBatch:
        batch_cont+=1
        model.predict_many(x,y,best_mask_index,task,mask_selection=False)
        model.learn_many(x,y,best_mask_index)
    model.save_final_metrics(task,best_mask_index)
    model.final_weights_saving()
  model.plotting()


file_path = f'Performance/Results/cPB/{dataset}/pretrain-T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10_iter{iteration}.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(model.performance, file)

file_path = f'Performance/Results/cPB/{dataset}/pretrain-T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10_iter{iteration}_selectedMask.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(model.selected_mask_index, file)


In [6]:
base_model = 'GRU'
batch_size = 128
hidden_size = 250
seq_len = 11
epoch_size=10
lr = 0.01
number_of_tasks=4
mask_selection_NofBatch = 50
input_size=2
iteration=1
mask_init='uniform'
dataset='Weather'
dataset_name = "weather_st124_4conf"
Pretrain_task = '1'

df = pd.read_csv(os.path.join(f"datasets/{dataset}/", f"{dataset_name}.csv"))
pretrain_model_addr=f'Performance/Pretrain/GRU/{dataset}/After/weather-{base_model}-pretrain-hidden{hidden_size}-epoch10_itter{iteration}.pickle'
mask_weights=[] #if we have initial masks then reload it here



In [7]:
model=cPB(lr = lr ,hidden_size=hidden_size, seq_len=seq_len,base_model=base_model, pretrain_model_addr=pretrain_model_addr,
           mask_weights=mask_weights, mask_init=mask_init, number_of_tasks=number_of_tasks,epoch_size=epoch_size,input_size=input_size)


for iter in range(0,iteration):
  # loop for each task
  for task in range(1, df["task"].max() + 1):
    df_task = df[df["task"] == task]
    df_task = df_task.drop(columns="task")
    # loop based on each batch of data
    batch_cont=0
    model.weights_copy(task)
    for i in range(0, len(df_task), batch_size):
      x = df_task.iloc[i : i + batch_size, 0:-1].values.astype(np.float32)
      y = list(df_task.iloc[i : i + batch_size, -1])
      if batch_cont<mask_selection_NofBatch:
        for mask_index in range (0, task):
          model.predict_many(x,y,mask_index,task,mask_selection=True)
          model.learn_many(x,y,mask_index)
        batch_cont+=1
      elif batch_cont==mask_selection_NofBatch:
        best_mask_index=model.add_new_column(task)
        model.predict_many(x,y,best_mask_index,task,mask_selection=False)
        model.learn_many(x,y,best_mask_index)
        batch_cont+=1
      elif batch_cont>mask_selection_NofBatch:
        batch_cont+=1
        model.predict_many(x,y,best_mask_index,task,mask_selection=False)
        model.learn_many(x,y,best_mask_index)
    model.save_final_metrics(task,best_mask_index)
    model.final_weights_saving()
  model.plotting()


file_path = f'Performance/Results/cPB/{dataset}/pretrain-T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10_iter{iteration}.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(model.performance, file)

file_path = f'Performance/Results/cPB/{dataset}/pretrain-T{Pretrain_task}_{base_model}-{dataset_name}-hidden{hidden_size}-epoch10_iter{iteration}_selectedMask.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(model.selected_mask_index, file)


RuntimeError: Error(s) in loading state_dict for GRU_Model:
	size mismatch for gru.weight_ih_l0: copying a param with shape torch.Size([750, 4]) from checkpoint, the shape in current model is torch.Size([750, 2]).

# Weather datasets


In [9]:
# TODO
dataset='Weather'
dataset_name = "weather_pretraining"
batch_size = 128
hidden_size = 250
seq_len = 11
iterations = 10
output_size = 2
num_layers = 1
loss_on_seq = False
freeze_inputs_weights = False
pretraining_samples = 0
pretraining_epochs = 0
write_weights = False
combination = False
model_type = 'GRU' # or LSTM
if model_type == "GRU":
    model_class = GRU_Model
elif model_type == "LSTM":
    model_class = LSTM_Model




In [10]:
df = pd.read_csv(f"datasets/{dataset}/{dataset_name}.csv")
df_task = df.drop('task', axis=1)
df_task

,RH,T_d,w_s,w_d,target
0,-0.878049,0.180556,0.142857,-1.333083,1
1,-0.878049,0.138889,0.571429,0.652632,1
2,-0.926829,0.111111,0.357143,-1.298496,1
3,-0.975610,0.125000,0.428571,-0.875188,1
4,-0.926829,0.138889,0.142857,-1.347368,1
...,...,...,...,...,...
24100,-0.707317,1.277778,0.428571,-1.346617,1
24101,0.560976,2.138889,0.714286,-0.121053,1
24102,-0.073171,2.041667,-0.142857,-0.309023,1
24103,-0.390244,1.722222,0.214286,-0.128571,1


In [ ]:
for itter in range(1,iterations+1):
  Model = model_class(input_size=4,
        device=torch.device("cpu"),
      	num_layers=num_layers,
        hidden_size=hidden_size,
        output_size=output_size,
        batch_size=batch_size,
          )
  Preprocess_object = Preprocess(seq_len=seq_len)
  with open(f"Performance/Pretrain/{model_type}/{dataset}/Before/{dataset}-{model_type}-pretrain-hidden{hidden_size}-epoch10_itter{itter}.pickle", "wb") as fp:
    pickle.dump(Model.state_dict(), fp)

  if len(df_task) % batch_size == 0:
    n_batches = int(len(df_task) / batch_size)
  else:
    n_batches = int(len(df_task) / batch_size) + 1
  optimizer = torch.optim.Adam(Model.parameters(), lr=0.01)
  loss_fn = torch.nn.CrossEntropyLoss(reduction="mean")
  out_h = None
  for j in range(0,10):
    for i in range(0, len(df_task), batch_size):
      x = df_task.iloc[i : i + batch_size, 0:-1].values.astype(np.float32)
      y = list(df_task.iloc[i : i + batch_size, -1])
      if len(y) >= seq_len:
        x = np.array(x)
        y = list(y)
        x, y, _ = Preprocess_object._load_batch(x, y)
        y_pred = Model(x)
        y_pred = get_samples_outputs(y_pred)
        pred, _ = get_pred_from_outputs(y_pred)
        loss = loss_fn(y_pred, y)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        # update weights
        optimizer.step()
  with open(f"Performance/Pretrain/{model_type}/{dataset}/After/{dataset}-{model_type}-pretrain-hidden{hidden_size}-epoch10_itter{itter}.pickle", "wb") as fp:
    pickle.dump(Model.state_dict(), fp)
  print(f'itter {itter} finished')

# SINE datasets


In [16]:
# TODO
dataset='SINE'
dataset_name = "sine_rw10_mode5_extended_6-6_1234"
Task_Number = 1
batch_size = 128
hidden_size = 250
seq_len = 10
iterations = 10
output_size = 2
num_layers = 1
loss_on_seq = False
freeze_inputs_weights = False
pretraining_samples = 0
pretraining_epochs = 0
write_weights = False
combination = False
model_type = 'GRU' # or LSTM
if model_type == "GRU":
    model_class = GRU_Model
elif model_type == "LSTM":
    model_class = LSTM_Model


In [17]:
df = pd.read_csv(f"datasets/{dataset}/{dataset_name}.csv")
df_task = df[df["task"] == Task_Number]
df_task = df_task.drop('task', axis=1)
df_task

,x1,x2,target
0,0.718354,0.957244,0
1,0.725173,0.952800,0
2,0.772454,0.922077,0
3,0.803595,0.872254,0
4,0.148729,0.872254,0
...,...,...,...
49995,0.431281,0.891558,0
49996,0.391986,0.877517,0
49997,0.417829,0.916618,0
49998,0.115487,0.916618,0


In [ ]:
for itter in range(1,iterations+1):
  Model = model_class(input_size=4,
        device=torch.device("cpu"),
      	num_layers=num_layers,
        hidden_size=hidden_size,
        output_size=output_size,
        batch_size=batch_size,
          )
  Preprocess_object = Preprocess(seq_len=seq_len)
  with open(f"Performance/Pretrain/{model_type}/{dataset}/Before/{dataset}-{model_type}-pretrain-hidden{hidden_size}-epoch10_itter{itter}.pickle", "wb") as fp:
    pickle.dump(Model.state_dict(), fp)

  if len(df_task) % batch_size == 0:
    n_batches = int(len(df_task) / batch_size)
  else:
    n_batches = int(len(df_task) / batch_size) + 1
  optimizer = torch.optim.Adam(Model.parameters(), lr=0.01)
  loss_fn = torch.nn.CrossEntropyLoss(reduction="mean")
  out_h = None
  for j in range(0,10):
    for i in range(0, len(df_task), batch_size):
      x = df_task.iloc[i : i + batch_size, 0:-1].values.astype(np.float32)
      y = list(df_task.iloc[i : i + batch_size, -1])
      if len(y) >= seq_len:
        x = np.array(x)
        y = list(y)
        x, y, _ = Preprocess_object._load_batch(x, y)
        y_pred = Model(x)
        y_pred = get_samples_outputs(y_pred)
        pred, _ = get_pred_from_outputs(y_pred)
        loss = loss_fn(y_pred, y)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        # update weights
        optimizer.step()
  with open(f"Performance/Pretrain/{model_type}/{dataset}/After/{dataset}-{model_type}-pretrain-hidden{hidden_size}-epoch10_itter{itter}.pickle", "wb") as fp:
    pickle.dump(Model.state_dict(), fp)
  print(f'itter {itter} finished')